# OpenAI Assistant Crafting Email Responses

The [OpenAI Assistant API (beta)](https://platform.openai.com/docs/assistants/overview) leverages existing [OpenAI products](https://platform.openai.com/docs/assistants/tools), such as storage, vector databases, code interpreter, and function calling, for AI applications. For example, users can create a vector database without coding or upload data to execute Python commands during a conversation. The Assistant API can also store user messages in threads and automatically truncate text when needed.

In this demo, we will create a vector database using the OpenAI website interface to store professors' course syllabi, retrieve students' emails, and use the OpenAI Assistant to respond to student emails automatically.

## Store Credentials

Purchase an OpenAI API key and create an OpenAI Assistant. Store the credentials in a `config.ini` file as follows:

```
[openai]
api_key = <your API key>

[openai-assistant]
assistant_id = <your assistant id>
```

## Retrieve Credentials

In [ ]:
import configparser

config = configparser.ConfigParser(interpolation=None)
config.read('config.ini')

openai_api_key   = config['openai']['api_key']

assistant_id = config['openai-assistant']['assistant_id']

## OpenAI Assistant API

Retrieve the API key and assistant ID. The assistant can access the uploaded course syllabus and final project instructions. 

A function is created to answer student emails. Provide the sender's name and email content, and the assistant will generate a reply based on the syllabus and final project instructions.

In [ ]:
from openai import OpenAI


client = OpenAI(api_key=openai_api_key)

#provide the assisatne id bellow
assistant = client.beta.assistants.retrieve(assistant_id)


In [ ]:
def ask_assistant(question):
    thread = client.beta.threads.create()
    
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=question
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    
    while run.status != 'completed':
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    return messages.data[0].content[0].text.value

Test the function.

In [ ]:
answer = ask_assistant("name xuebin, body: when is the final exam ")
print(answer)

## Reply to Emails

This code runs locally to retrieve received emails in Outlook. If the sender's address matches specific patterns, the sender's name and message are passed to the OpenAI Assistant function. 

The automatically created reply will be saved as a draft.

In [ ]:
import win32com.client
import re

def extract_name(email):
    pattern = r"^([^@]+)@"
    match = re.search(pattern, email)
    return match.group(1) if match else email

def process_emails(sender_filter):
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.GetDefaultFolder(6)
    messages = inbox.Items
    messages.Sort("[ReceivedTime]", True)

    for message in messages:
        if sender_filter.lower() in message.SenderEmailAddress.lower():
            if message.UnRead :
                sender_name = message.SenderName
                body_text = message.Body
    
                print(f"Sender: {sender_name}")

                print(f"Body: {body_text[:100]}...")  # Print first 100 characters
    
                reply = message.Reply()
                reply_body = ask_assistant(f"sender: {sender_name}, body:{body_text}")
                reply.Body = f"{reply_body}\n\n--- Original Message ---\n{message.Body}"
              
                reply.Save()

    


Test emails

In [ ]:
sender_filter = "weixuebin@gmail.com"  # Replace with the desired sender domain or email
process_emails(sender_filter)